In [79]:
import pandas as pd

In [80]:
def print_full(df):
    pd.set_option('display.max_colwidth', 2000)
    # pd.set_option('display.width', 2000)

    print(df)
    # pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 2000)
# pd.set_option('display.float_format', '{:20,.2f}'.format)
# pd.set_option('display.max_colwidth', None)
# print(x)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')
# pd.reset_option('display.float_format')
# pd.reset_option('display.max_colwidth')

In [90]:
df = pd.read_csv("contrib_from_csv.csv", sep = ",", encoding = "utf-8", dtype= str)

if df.isnull().values.any() :
    print("y a t il des nan? ->",df.isnull().values.any())
    print("combien y a t il de nan? ->",df.isnull().values.sum())
    print("où sont les null? ->\n",df.isnull().sum())
    df[df.isnull().T.any()]
else :
    print("il n'y a pas de NaN")

il n'y a pas de NaN


In [91]:
# df["joined_id"] = df["joined_id"].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))

In [92]:
print(df.dtypes.nunique()>1)

df._is_mixed_type

for column in df.columns:
    print(pd.api.types.infer_dtype(df[column]))

False
string
string
string
string
string
string
string
string
string
string


In [93]:
print_full(df[2950 :2952])

     Catégorie Date de réception Code postal Code INSEE   
2950        CC            190322       47200      47157  \
2951        CC            190322       47200      47157   

     Numéro d'ordre arbitraire Type Graphie TT Numéro de page   
2950                     18623              MD             32  \
2951                     18623              MD             32   

     Numéro séquentiel   
2950                80  \
2951                81   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [94]:

# # trace de illisibles

# # supprimer les illisibles de ce corpus (plein de types de codages différents)
kill_texte_illisible = lambda x : x.replace("texte illisible", " ")
kill_illisible_mot = lambda x : x.replace("[mot ]", " ")
soupir = lambda x : x.replace("[mot illisible]", " ")
soupir2 = lambda x : x.replace("[mots illisibles]", "")
kill_mot_illisble = lambda x : x.replace("mot illisible", " ")
kill_illisible = lambda x : x.replace("illisible", " ")
kill_illisible_plural = lambda x : x.replace("illisibles", " ")
kill_illisible_capslock = lambda x : x.replace("ILLISIBLE", " ")
kill_illisibleS_capslock = lambda x : x.replace("ILLISIBLES", " ")

df["clean"] = df["Contribution"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible_mot)
df["clean"] = df["clean"].apply(soupir)
df["clean"] = df["clean"].apply(soupir2)
df["clean"] = df["clean"].apply(kill_mot_illisble)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)
df["clean"] = df["clean"].apply(kill_illisible_capslock)
df["clean"] = df["clean"].apply(kill_illisibleS_capslock)

df["clean"] = df["clean"].replace("^il(?:l(?:(?:(?:isuble|sibil)|isubl)|sible)|lisible|ilsible|li?sble)$", " ", regex = True)

# supprimer les fins de lignes restantes
join_last_breaklines = lambda x : x.replace("\n"," ") 

# application des fonctions de nettoyage 
# reconstruition des phrases
# en cas d'item suivi d'une majuscule, remplace par un point + majuscule, en cas de minuscule, espace + minuscule.
df["clean"] = df["clean"].replace({'\n((\s?)[A-Z]+)' : ". \\1"}, regex = True)
df["clean"] = df["clean"].replace({"\x0b(( ?)[A-Z])" : ". \\1"}, regex = True)
df["clean"] = df["clean"].replace({"x92(( ?)[A-Z])" : '. \\1'}, regex = True)
df["clean"] = df["clean"].replace({"\v(( ?)[A-Z])" : '. \\1'}, regex = True)

df["clean"] = df["clean"].replace({"\v(( ?)[a-z])" : ' \\1'}, regex = True)
df["clean"] = df["clean"].replace({"\n([a-z])" : " \\1"}, regex = True)
df["clean"] = df["clean"].replace({"\x0b([a-z])" : " \\1"}, regex = True)
df["clean"] = df["clean"].replace({"x92([a-z])" : " \\1"}, regex = True)

df["clean"] = df["clean"].replace({"x92([a-z])" : " \\1"}, regex = True)

df["clean"] = df['clean'].apply(join_last_breaklines)
df["clean"] = df["clean"].replace({"\.+" : " . "}, regex = True)
df["clean"] = df["clean"].replace({"\s+" : " "}, regex = True)


# nettoyage des mails
df["clean"] = df["clean"].replace('(\w\-?)+@(\w\-?)+[.]{1}[a-z]+', " ", regex = True)
# ((\w+(-?)|(\.?))+|(\w+)@(\w\-?)+[.]{1}[a-z]+)
# nettoyage des sites
df["clean"] = df["clean"].replace('(http:\/\/|https:\/\/|www.)((\w)+(\/?)(\w+))+(\.?)+((\w)+(\/?)(\w+))+', " ", regex = True)

# nettoyage des chiffres et dates? ça contient des informations aussi, 80km par ex.
# df["clean"] = df["clean"].replace('[0-9]+(\/?)[0-9]+(\/?)[0-9]+', "", regex = True)

# nettoyage des lambda restants
df["clean"] = df["clean"].replace('(\\\|\/)', " ", regex = True)

# trace de mots récurrents sans contenus pertinents
mots_a_clean = ["fin page", "page écrire", "wanadoo", "vierge rayer", "page vierge", "text area", "gmail.com", "neuillyplaisance", "cordialement","bonjour", "au revoir"]
for mot in mots_a_clean :
    that = lambda x : x.lower().replace(mot, " ")
    df["clean"] = df["clean"].apply(that)
df["clean"] = df["clean"].replace('■'," ")
# ■ n'a pas de code ASCII, cette fonction n'a probablement pas d'effet
df["clean"] = df["clean"].replace({"([i|I][s|S][f|F])" : " \\1 "}, regex = True)

df["clean"] = df["clean"].replace("<image>", " ")


In [95]:
print_full(df["clean"].sample(50))

608                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        de plus dans le contexte actuel il faut avec rigueur distinguer immigration et invasion . il ne fait aucun doute qu'il existe des « passeurs », dans quel but sont-ils recrutés ? à quel niveau sont-ils rémunérés . qui, en fait, active le phénomène migrant qui est un déracinement, un appauvrissement, du pays d'émigration . que n'a-t-on pas fait pour que ces pays accèdent à un juste niveau de progrès et à une juste aisance i . ne faut-il pas les aider à se dév

In [96]:
# écriture du résultat dans un csv
df.to_csv("df_CC_cleaned.csv", index = False, sep = ",", encoding = "utf-8")


In [97]:
df = pd.read_csv("df_CC_cleaned.csv", sep = ",", encoding = "utf-8", dtype=str)

In [98]:
for column in df.columns:
    print(pd.api.types.infer_dtype(df[column]))


if df.isnull().values.any() :
    print("y a t il des nan? ->",df.isnull().values.any())
    print("combien y a t il de nan? ->",df.isnull().values.sum())
    print("où sont les null? ->\n",df.isnull().sum())
    print(df[df.isnull().T.any()])
else :
    print("il n'y a pas de NaN")
    
print(df[column][df[column].apply(lambda x: isinstance(x, type))])
# df["joined_id"] = df["joined_id"].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))
print(df._is_mixed_type)
print(df.dtypes.nunique()>1)
# In [3600]: d2._is_mixed_type
# In [3602]: d1.dtypes.nunique()>1
print(df.dtypes)

string
string
string
string
string
string
string
string
string
string
string
il n'y a pas de NaN
Series([], Name: clean, dtype: object)
False
False
Catégorie                    object
Date de réception            object
Code postal                  object
Code INSEE                   object
Numéro d'ordre arbitraire    object
Type Graphie TT              object
Numéro de page               object
Numéro séquentiel            object
Contribution                 object
joined_id                    object
clean                        object
dtype: object
